In [ ]:
print('Starting imports')

import requests
import json
import pandas as pd
import time

endpoint = 'https://services2.arcgis.com/HdTo6HJqh92wn4D8/arcgis/rest/services/Metro_Nashville_Police_Department_Active_Dispatch_Table_view/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

i = 0
t = 10 # start a 10-second countdown.  For fun!!

print('Starting loop')
print('')

while True:

    t -= 1

    if t == 0 :
        # process it here!
        try :
            response = requests.get(endpoint)
            if(response.status_code == 200) :
                res = response.json()
                active_dispatch = pd.DataFrame(columns = ['incident_type_code','incident_type_name','call_received_time','location',
                                                 'location_description','city_name','last_updated'])
                for incident in res['features'] :
                    new_row = {'incident_type_code': incident['properties']['IncidentTypeCode'],
                               'incident_type_name': incident['properties']['IncidentTypeName'],
                               'call_received_time': incident['properties']['CallReceivedTime'],
                               'location': incident['properties']['Location'],
                               'location_description': incident['properties']['LocationDescription'],
                               'city_name': incident['properties']['CityName'],
                               'last_updated': incident['properties']['LastUpdated']}
                    active_dispatch.loc[len(active_dispatch)] = new_row
                if len(active_dispatch) > 0 :
                    last_updated = active_dispatch.loc[0,'last_updated']
                    active_dispatch.to_csv('../data/active_dispatch/' + str(last_updated) + '.csv', index=False)
                    i += 1
                t = 360          # set the next countdown to 360 seconds, 5 minutes later
            else :
                print('')
                print('Error!  Response : ' + str(response))
                t = 60          # try again in 1 minute
        except requests.exceptions.RequestException as e:
            print('')
            print('Error!  Requests error : ' + str(e))
            t = 60          # try again in 1 minute
        except:
            print('')
            print('Something else went wrong') 
            t = 60          # try again in 1 minute
    print(f"\rProcessed: {i}.  Seconds until next process: {t}    ", end = "")
    time.sleep(1)              # count down every second!

Starting imports
Starting loop

Processed: 238.  Seconds until next process: 249    